# <font color='green'> Vamos a implementar un clasificador de si habrá lluvia (salida binaria), y de probabilidad de lluvia (predict_proba) <font>

 Constructores de nuestro dataframe: <p>
    - build_data_frame_struct: construye el contenedor de nuestros datos <p>
    - fill_prediction_model_dataframe: rellena la estructura con los registros pasados mediante 'query_result' <p>
    - build_dataframe: construye el dataframe final con los datos obtenidos de la query a la ontología <p> 

In [4]:
def build_data_frame_struct(columns_series, column_name_index=None):
    import pandas as pd
    try:
        df_struct = pd.DataFrame(columns=columns_series)
        if column_name_index is not None:
            df_struct.set_index(column_name_index, inplace=True)
    except Exception as exc:
        print('excepción surgida: {}'.format(exc))
    
    return df_struct

def fill_prediction_model_dataframe(final_dataframe_struct, query_result):
    import pandas as pd
    from tqdm import tqdm
    try:
        final_dataframe = final_dataframe_struct
        for element in tqdm(query_result):
            #for element in query_result:
            pred_model_dict = element['predictionmodel']

            final_dataframe=final_dataframe.append(pd.Series(pred_model_dict), ignore_index=True)
        return final_dataframe    
    except Exception as exc:
          print(exc)           

def build_dataframe(client_object, _ontology, columns_names, limit_rows_number=10000): # start_datetime=None, end_datetime=None):
  import pandas as pd
  import time    

  _ok_equipment = False 
  _res_equipment = [] 
  select_query = ''
  try:
    #iniciamos nuestro dataframe con la estructura, sin valores:
    final_df = build_data_frame_struct(columns_names)
    select_query = 'select * from ' + _ontology + ' as c limit ' + str(limit_rows_number)
    #print(select_query_equipment_normalidad)
    client_object.connect()
    _ok, _res_from_query = client_object.query(query=select_query, ontology=_ontology , query_type="SQL")
    print(_res_from_query)
    res_from_query_df_struct = build_data_frame_struct(columns_names)
    df_query_result = fill_prediction_model_dataframe(res_from_query_df_struct, _res_from_query)
    final_df = pd.DataFrame(df_query_result)
    client_object.leave()
    #time.sleep(2)
    return final_df
  except Exception as exc:
    print('excepción surgida: {}'.format(exc))


 Instanciamos nuestro objeto client para conexión a datos:

In [5]:
from onesaitplatform.iotbroker import IotBrokerClient
client = IotBrokerClient(host="<host>",
                        iot_client="<iot_client>", 
                         iot_client_token="<iot_client_token>")
client.protocol = "http" #"https"
client.avoid_ssl_certificate = True


In [6]:
_ontology = "predictionmodel"
columns_names = ['tmp0', 'tmp1', 'hPa', 'hum', 'pp']
limit_rows_number = 50000
_res_df = build_dataframe(client, _ontology, columns_names, limit_rows_number)


  0%|          | 0/50000 [00:00<?, ?it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 50000/50000 [03:06<00:00, 267.60it/s]


 Check dataframe content:

In [8]:
def check_dataframe_content(dataframe):
    try:
        df_length = len(dataframe)
        head_df = dataframe.head(5)
        tail_df = dataframe.tail(5)
    except Exception as exc:
        print('excepción surgida: {}'.format(exc))

    return df_length, head_df, tail_df

res_df_length, res_df_head_df, res_df_tail_df = check_dataframe_content(_res_df)

In [9]:
print('dataframe length: {}'.format(res_df_length))
res_df_head_df.style


dataframe length: 50000


,tmp0,tmp1,hPa,hum,pp
0,39.89,51.48,1092,0.93,0
1,7.2,7.01,936,0.57,0
2,18.55,20.12,981,0.66,0
3,16.77,16.77,1053,0.61,0
4,41.3,35.17,1018,0.14,0


- Visualización de atributos en formato de serie temporal:
- Visualización de atributos en formato histograma
- Check: rango de valores 
- Check: posibles outliers

In [11]:
def plot_series_values(dataframe, attribute, figsize_rows=15, figsize_cols=10):
    import matplotlib.pyplot as plt
    plt.subplots(figsize=(figsize_rows, figsize_rows))  
    plt.plot(dataframe[attribute].values)  

    plt.show()
    
    return 

def plot_series_histogram(dataframe, attribute, figsize_rows=15, figsize_cols=10):
    import matplotlib.pyplot as plt
    plt.subplots(figsize=(figsize_rows, figsize_rows))
    plt.hist(dataframe[attribute])

    plt.show()
 
def check_values_range(dataframe, attribute, expected_range_min, expected_range_max):
    try:
        below_range_mask = dataframe[attribute].values < expected_range_min
        above_range_mask = dataframe[attribute].values > expected_range_max
        below_range_samples = dataframe[below_range_mask]
        above_range_samples = dataframe[above_range_mask]

        return below_range_samples, above_range_samples
    except Exception as exc:
        print('excepción surgida: {}'.format(exc))

def get_outliers_tukey_test(dataframe, attribute):
    try:
        import numpy as np 
        mask_not_nan_values = np.isnan(dataframe[attribute].values)==False
        not_nan_values = dataframe[attribute][mask_not_nan_values]

        q1 = np.percentile(not_nan_values, 25)
        q3 = np.percentile(not_nan_values, 75)
        iqr = q3 - q1 
        print("First quartile, third quartile and interquartile range: [q1=%f, q3=%f, iqr=%f]" % (q1, q3, iqr))
        
        floor = q1 - 1.5*iqr
        ceiling = q3 + 1.5*iqr
        print("[floor=%f, ceiling=%f]" % (floor, ceiling))
        
        outlier_indices = list(dataframe[attribute].index[(dataframe[attribute].values < floor)|(dataframe[attribute].values > ceiling)])  
        samples_with_outlier_values = dataframe.iloc[outlier_indices]

        return outlier_indices, samples_with_outlier_values
    except Exception as exc:
        print('excepción surgida: {}'.format(exc))


 Ejemplo de check de outlier values para la variable 'tmp0':

In [17]:
attribute_outlier_indices, attribute_samples_with_outlier_values = get_outliers_tukey_test(_res_df, 'tmp0')
print('tmp0 outlier indices: {}'.format(attribute_outlier_indices))
if any(attribute_outlier_indices):
    print(' tmp0 outlier values: {}'.format(attribute_samples_with_outlier_values))


First quartile, third quartile and interquartile range: [q1=11.427500, q3=34.130000, iqr=22.702500]
[floor=-22.626250, ceiling=68.183750]
tmp0 outlier indices: []


 Missing values in attribute check

In [18]:
def check_for_missing_values(dataframe, attribute):
    try:
        import numpy as np
        nan_values_mask = np.isnan(dataframe[attribute])==True
    
        return dataframe[nan_values_mask] 

    except Exception as exc:
        print('excepción surgida: {}'.format(exc))

# Interpolation of missing values
'''
    - desired_strategy: method to impute missing values (https://scikit-learn.org/stable/modules/impute.html)
'''
def impute_attribute(dataframe, attribute, desired_strategy):
    try:
        from sklearn.preprocessing import Imputer
        import numpy as np

        attribute_values = dataframe[attribute].values.reshape(-1, 1)
        imp = Imputer(missing_values=np.nan, strategy=desired_strategy, axis=0)
        imp.fit(values_)
        transformed_values = imp.transform(attribute_values)
        dataframe.loc[:,attribute] = transformed_values
        return dataframe
    except Exception as exc:
        print('excepción surgida: {}'.format(exc))



Models training functions
- split_into_train_test_sets: dividir el dataset en train y test sets via single split
- fit_dummy_classifier: baseline predictor, o modelo que predice con reglas simples, cuyo score nos sirve como mínimo umbral para las predicciones de los modelos entrenados; wrapper de scikit-learn: https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html
- select_model_via_grid_with_cross_validation: 

In [19]:
def split_into_train_test_sets(dataframe, attributes, target, test_fraction, random_state=None):
    try:
        from sklearn.model_selection import train_test_split

        X_train, X_test, y_train, y_test = train_test_split(dataframe[attributes], dataframe[target], test_size=test_fraction, random_state=random_state)
        return X_train, X_test, y_train, y_test
    except Exception as exc:
        print('excepción surgida: {}'.format(exc))

def fit_dummy_classifier(train_set_attributes, train_set_target, classifier_strategy, random_state=None):
    try:
        from sklearn.dummy import DummyClassifier
    
        dummy_classifier = DummyClassifier(strategy=classifier_strategy, random_state=random_state)
        dummy_classifier.fit(train_set_attributes, train_set_target)
        return dummy_classifier

    except Exception as exc:
        print('excepción surgida: {}'.format(exc))


 Entrenamos un baseline model:

In [20]:
X_train, X_test, y_train, y_test = split_into_train_test_sets(_res_df, _res_df.columns[:-1], _res_df.columns[-1], 0.3)
dummy_clf = fit_dummy_classifier(X_train, y_train, 'most_frequent')
# ejemplo de predicción:
dummy_clf.predict(y_test)
dummy_clf.predict(y_test.iloc[3].reshape(1,))

In [21]:
_res_df=None
# ejemplo de predicción:
dummy_clf.predict(y_test.iloc[3].reshape(1,))


array([0.])

Ante la posibilidad de entrenar un clasificador, tenemos varios posibles modelos, cada uno implementando un algoritmo <p>
con sus parámetros; para decidir qué algoritmo y qué parámetros podemos escoger, podemos realizar:
- gridsearch: para datasets relativamente pequeños o si no necesitamos entrenamientos ágiles en el tiempo, es una forma de <p>
"barrer" todas las posibles combinaciones de entre una lista de posibles valores de los parámetros indicados 
- para que las métricas obtenidas sean representativas, siempre realizar una validación cruzada en base a varios train-test splits, <p>
frente al comunmente utilizado train-test split, más info:  https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate

In [22]:
def select_model_via_grid_search_cv(models_list, models_params_dict, X_train, y_train, cv_folds=None, scoring_metrics=None, refit_metric=None):
    try:
        import sklearn
        from sklearn.model_selection import GridSearchCV
        from tqdm import tqdm

        cv_results_df = pd.DataFrame()
        best_estimators_dict = {} 
        i = 0
        for model_name, hyperparams in tqdm(models_params_dict.items()):
            print(model_name)
            model = sklearn.base.clone(models_list[i])
            clf = GridSearchCV(model, hyperparams, cv=cv_folds, scoring=scoring_metrics, refit=refit_metric, return_train_score=True) 
            clf.fit(X_train, y_train)
            
            cv_model_best_results=pd.DataFrame(columns=list(pd.DataFrame(clf.cv_results_).columns))
            cv_model_best_results.loc[model, :] = pd.Series(pd.DataFrame(clf.cv_results_).iloc[clf.best_index_]) 
            cv_results_df = cv_results_df.append(cv_model_best_results)

            best_estimators_dict[model_name] = clf.best_estimator_

            i += 1
        return cv_results_df, best_estimators_dict
    except Exception as exc:
        print('excepción surgida: {}'.format(exc))


 Conversión de variable objetivo a variable binaria

In [23]:
def binarize_target_variable(target_series):
    try: 
        binary_target_series_mask = target_series>0
        target_series[binary_target_series_mask] = 1
        
        return target_series
    except Exception as exc:
        print('excepción surgida: {}'.format(exc))

# Estandarización de valores de atributos
def standard_scaler_transformer(X_train_set):
    try:
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()

        scaled_train_df_values = scaler.fit_transform(X_train_set.values)
        scaled_train_df = pd.DataFrame(columns=X_train_set.columns, data=scaled_train_df_values)

        return scaled_train_df
    except Exception as exc:
        print('excepción surgida: {}'.format(exc))

X_train_scaled = standard_scaler_transformer(X_train)
X_train=None

 Entrenamos con 3 tipos de modelos clasificadores

In [24]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

models_and_params = {'GaussianNB': {'var_smoothing': [1e-09, 1e-08, 1e-10]}, 
                          'LogisticRegression': {'solver': ['liblinear'], 'penalty': ['l1', 'l2'], 'C': [1, 0.1, 0.01]},
                          'SVC': {'C': [1, 0.1, 0.01], 'gamma': ['scale', 'auto']}}

GaussianNB_clf = GaussianNB()
LogisticRegression_clf = LogisticRegression()
SVC_clf = SVC()
models_list = [GaussianNB_clf, LogisticRegression_clf, SVC_clf]

y_train_binarized = binarize_target_variable(y_train)
y_train=None

cv_results_df, best_estimators_dict = select_model_via_grid_search_cv(models_list, models_and_params, X_train_scaled, y_train_binarized.values, cv_folds=5, 
                                scoring_metrics=['recall', 'f1', 'roc_auc'], refit_metric='recall')

cv_results_df.style


 33%|███▎      | 1/3 [00:01<00:02,  1.08s/it]C:\Users\gcabreram\AppData\Local\Continuum\anaconda3\envs\test_virt_env\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
 67%|██████▋   | 2/3 [00:19<00:06,  6.25s/it]C:\Users\gcabreram\AppData\Local\Continuum\anaconda3\envs\test_virt_env\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
100%|██████████| 3/3 [08:52<00:00, 158.25s/it]


GaussianNB
LogisticRegression
SVC


,mean_fit_time,mean_score_time,mean_test_f1,mean_test_recall,mean_test_roc_auc,mean_train_f1,mean_train_recall,mean_train_roc_auc,param_C,param_gamma,param_penalty,param_solver,param_var_smoothing,params,rank_test_f1,rank_test_recall,rank_test_roc_auc,split0_test_f1,split0_test_recall,split0_test_roc_auc,split0_train_f1,split0_train_recall,split0_train_roc_auc,split1_test_f1,split1_test_recall,split1_test_roc_auc,split1_train_f1,split1_train_recall,split1_train_roc_auc,split2_test_f1,split2_test_recall,split2_test_roc_auc,split2_train_f1,split2_train_recall,split2_train_roc_auc,split3_test_f1,split3_test_recall,split3_test_roc_auc,split3_train_f1,split3_train_recall,split3_train_roc_auc,split4_test_f1,split4_test_recall,split4_test_roc_auc,split4_train_f1,split4_train_recall,split4_train_roc_auc,std_fit_time,std_score_time,std_test_f1,std_test_recall,std_test_roc_auc,std_train_f1,std_train_recall,std_train_roc_auc
"GaussianNB(priors=None, var_smoothing=1e-09)",0.0125992,0.0149793,0.717264,0.815013,0.963931,0.718419,0.816287,0.964086,nan,nan,nan,nan,1e-09,{'var_smoothing': 1e-09},1,1,1,0.726183,0.812332,0.96474,0.715529,0.813003,0.96403,0.732279,0.837802,0.967384,0.716948,0.812332,0.963451,0.712085,0.80563,0.961886,0.718544,0.82004,0.96447,0.706776,0.810992,0.964458,0.720944,0.819035,0.963933,0.708995,0.808311,0.961185,0.72013,0.817024,0.964546,0.0024214,0.0031592,0.0101015,0.0116244,0.00221688,0.00199184,0.00311788,0.000397288
"LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', random_state=None, solver='warn', tol=0.0001, verbose=0, warm_start=False)",1.74155,0.012399,0.93008,0.92815,0.998452,0.930655,0.928753,0.998458,1,nan,l1,liblinear,nan,"{'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}",1,1,1,0.927322,0.923592,0.998368,0.931341,0.929625,0.998477,0.917293,0.899464,0.998166,0.932775,0.932306,0.998531,0.927285,0.931635,0.998472,0.931341,0.929625,0.998451,0.93654,0.939678,0.998489,0.929364,0.925938,0.998451,0.941961,0.946381,0.998767,0.928451,0.926273,0.998378,0.0501849,0.00302549,0.00850666,0.0162547,0.000194892,0.00154634,0.00237534,4.94661e-05
"SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, decision_function_shape='ovr', degree=3, gamma='auto_deprecated', kernel='rbf', max_iter=-1, probability=False, random_state=None, shrinking=True, tol=0.001, verbose=False)",2.2935,0.941903,0.927492,0.919571,0.998228,0.929647,0.922453,0.998334,1,scale,nan,nan,nan,"{'C': 1, 'gamma': 'scale'}",1,1,2,0.921409,0.911528,0.998245,0.930469,0.921582,0.998306,0.918771,0.902145,0.997922,0.930743,0.923257,0.998427,0.92814,0.926273,0.998226,0.930773,0.925938,0.998331,0.926302,0.918231,0.998159,0.930256,0.925268,0.998349,0.942838,0.939678,0.998587,0.925995,0.91622,0.998257,0.0784337,0.0323878,0.0083712,0.0128014,0.000213428,0.00183595,0.00347363,5.57638e-05


 features scaling and target binarization:

In [25]:
X_test_scaled = standard_scaler_transformer(X_test)
X_test=None
y_test_binarized = binarize_target_variable(y_test)
y_test=None

- choose the best model based on its defined test metric
- 'test_metric' must match a sklearn scoring param: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [26]:
def evaluate_recall_on_test_set(best_estimators_dictionary, X_test, y_test, average='binary'):
    try:
        from sklearn.metrics import recall_score 

        test_metric_results_df = pd.DataFrame(columns=['model_name', 'recall_score'])
        for estimator_name, estimator in best_estimators_dictionary.items():
            estimator_dict = {'model_name': estimator_name, 'recall_score': None}
            y_pred = estimator.predict(X_test) 
            recall_score_value = recall_score(y_test, y_pred, average=average)
            estimator_dict['recall_score'] = recall_score_value
            test_metric_results_df = test_metric_results_df.append(pd.Series(estimator_dict), ignore_index=True)

        return test_metric_results_df

    except Exception as exc:
        print('excepción surgida: {}'.format(exc))

In [27]:
best_estimators_dict 

{'GaussianNB': GaussianNB(priors=None, var_smoothing=1e-09),
 'LogisticRegression': LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='warn',
           n_jobs=None, penalty='l1', random_state=None, solver='liblinear',
           tol=0.0001, verbose=0, warm_start=False),
 'SVC': SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False)}

- escogemos el mejor modelo en base a la sensibilidad (sensitivity o recall) de nuestro modelo para detectar verdaderos positivos (esto es, cuando hay lluvia) <p>
- además, agregamos las predicciones del baseline model, que nos sirve de referencia como umbral a superar (i.e. nuestro modelo final no puede empeorar a nuestro dummy_classifier)

- escogemos el mejor modelo en base a la sensibilidad (sensitivity o recall) de nuestro modelo para detectar verdaderos positivos (esto es, cuando hay lluvia) <p>
- además, agregamos las predicciones del baseline model, que nos sirve de referencia como umbral a superar (i.e. nuestro modelo final no puede empeorar a nuestro dummy_classifier)

In [29]:
best_estimators_dict['baseline_predictor'] = dummy_clf
test_metric_results_df = evaluate_recall_on_test_set(best_estimators_dict, X_test_scaled, y_test_binarized)
test_metric_results_df

,model_name,recall_score
0,GaussianNB,0.786232
1,LogisticRegression,0.904589
2,SVC,0.891304
3,baseline_predictor,0.000000


 Escogemos el mejor estimador en base a la métrica elegida, en este caso el recall:

In [30]:
def choose_best_estimator(validation_results_dataframe, metric, best_estimators_dictionary):
    try:
        max_metric_score = validation_results_dataframe[metric].max()
        max_metric_score_mask = validation_results_dataframe[metric]==max_metric_score
        best_estimator_df = validation_results_dataframe[max_metric_score_mask]
        best_estimator_name = best_estimator_df.iloc[0]['model_name']

        best_estimator_object = best_estimators_dictionary[best_estimator_name]

        return best_estimator_df, best_estimator_object 
    except Exception as exc:
        print('excepción surgida: {}'.format(exc))

best_estimator_df, best_estimator_object = choose_best_estimator(test_metric_results_df, 'recall_score', best_estimators_dict)

Predictions:
  * lluvia/no lluvia
  * probabilidad de lluvia/no lluvia 

In [32]:
def create_predictions_dataframe(X_data, final_estimator):
    try:
        import pandas as pd
        import numpy as np
        
        rain_preds_df = pd.DataFrame(columns=list(X_data.columns)+['prediction', 'prediction_probability'])
        for index, sample in X_data.iterrows():
            rain_preds_df.loc[index, 'tmp0'] = sample['tmp0']
            rain_preds_df.loc[index, 'tmp1'] = sample['tmp1']
            rain_preds_df.loc[index, 'hPa'] = sample['hPa']
            rain_preds_df.loc[index, 'hum'] = sample['hum']
            rain_preds_df.loc[index, 'prediction'] = final_estimator.predict(np.array(sample).reshape(1, -1))
            rain_preds_df.loc[index, 'prediction_probability'] = final_estimator.predict_proba(np.array(sample).reshape(1, -1)) 

        return rain_preds_df
    except Exception as exc:
        print('excepción surgida: {}'.format(exc))
        
rain_preds_df_ = create_predictions_dataframe(X_test_scaled.iloc[:10], best_estimator_object)    

rain_preds_df_

,tmp0,tmp1,hPa,hum,prediction,prediction_probability
0,1.17963,1.66536,1.64022,1.66729,[0.0],"[[0.9998139031181318, 0.00018609688186826598]]"
1,-0.720244,-0.580321,-0.607262,0.873119,[0.0],"[[0.9999993159087687, 6.840912313102597e-07]]"
2,-0.317101,-0.431182,-0.998869,-0.78427,[0.0],"[[1.0, 2.83409169365457e-20]]"
3,-0.9603,-1.01913,-0.52213,-0.680683,[0.0],"[[1.0, 1.9415543621508985e-19]]"
4,1.01197,0.611355,-1.23724,1.21841,[1.0],"[[0.01139401438361165, 0.9886059856163883]]"
5,-1.60502,-1.48161,-0.266734,-0.818799,[0.0],"[[1.0, 2.4577626940568113e-22]]"
6,1.33586,0.503803,0.584587,0.52783,[0.0],"[[0.5737165034476223, 0.42628349655237774]]"
7,1.46694,2.24041,0.107848,-0.231807,[0.0],"[[1.0, 4.195399064026205e-26]]"
8,1.18116,1.06021,1.64022,-0.0936908,[0.0],"[[0.9999999999999875, 1.255370530665657e-14]]"
9,0.635505,0.427083,-0.385918,-0.542567,[0.0],"[[1.0, 1.4068480881107985e-17]]"
